In [3]:
from IPython.display import display, HTML
display(HTML("""
<style>
div.container{width:90% !important;}
div.cell.code_cell.rendered{width:100%;}
div.input_prompt{padding:0px;}
div.CodeMirror {font-family:Consolas; font-size:12pt;}
div.text_cell_render.rendered_html{font-size:12pt;}
div.output {font-size:12pt; font-weight:bold;}
div.input {font-family:Consolas; font-size:12pt;}
div.prompt {min-width:70px;}
div#toc-wrapper{padding-top:120px;}
div.text_cell_render ul li{font-size:12pt;padding:5px;}
table.dataframe{font-size:12px;}
</style>
"""))

<b><font size="6" color="red">04_tensorflow_v1_tensor객체흐름</font></b>

# 1. tensorflow v2.xx에서 v1 사용하기

In [5]:
import tensorflow.compat.v1 as tf

In [6]:
tf.disable_v2_behavior() # tensorflow V2 비활성화하고 v1활성화
import numpy as np
import pandas as pd

Instructions for updating:
non-resource variables are not supported in the long term


# Tensorflow
 - 데이터 흐름 그래프(tensor 객체의 흐름)을 사용하는 수치 계산 라이브러리
 - 그래프는 node(연산)와 edge로 구성
 - sess = tf.Session()을 이용해서 실행 환경
 - sess.run()을 통해서 실행 결과를 확인

In [10]:
# 1단계 : tensor(상수 노드 1개) 정의
node1 = tf.constant('Hello, Tensorflow')
# 2단계 : 세션(연산을 실행하는 환경) 생성
sess = tf.Session()
# print(node1) # 아직 없음
# 3단계 : 실제 실행
print(sess.run(node1))
print(sess.run(node1).decode())

b'Hello, Tensorflow'
Hello, Tensorflow


In [12]:
# 간단한 연산 tensor 그래프
# 1. 그래프 정의
node1 = tf.constant(10, dtype=tf.float16)
node2 = tf.constant(20, dtype=tf.float16)
node3 = tf.add(node1, node2)
# 2. 세션 생성
sess = tf.Session()
print(sess.run([node1, node2, node3]))

[10.0, 20.0, 30.0]


In [13]:
# 타입변경
import numpy as np
node1 = tf.constant(np.array([1,2,3]), dtype=tf.int16)
node2 = tf.cast(node1, dtype=tf.float32)
sess = tf.Session()
print(sess.run([node1, node2]))

[array([1, 2, 3], dtype=int16), array([1., 2., 3.], dtype=float32)]


In [14]:
# 평균값 계산 : tf.reduce_mean()
data = np.array([1.,2.,3.,4.])
m = tf.reduce_mean(data)
sess = tf.Session()
print(sess.run(m))

2.5


In [15]:
# tf.random_normal([size]) : 평균이 0이고 표준편차가 1인 난수 size개 발생
w = tf.random_normal([3]) 
sess = tf.Session()
sess.run(w)

array([ 1.6073271,  0.4279549, -0.6173332], dtype=float32)

In [16]:
#  변수 노드
w = tf.Variable(tf.random_normal([1]))
sess = tf.Session()
sess.run(tf.global_variables_initializer()) # 변수 초기화
sess.run(w)

array([-0.51092505], dtype=float32)

# 2. tensorflow v1을 이용한 회귀분석 구현
## 2.1 독립변수 x가 1개, 타겟(종속)변수 y가 1개

In [17]:
# tensor 그래프 정의
# 데이터 셋 확보
x = np.array([1,2,3])
y = np.array([2,3,4])
# weight와 bias
W = tf.Variable(tf.random.normal([1]), name='weight')
b = tf.Variable(tf.random.normal([1]), name='bias')
# Hat, Hypothesis : 결과는 numpy 배열
H = W * x + b
# cost function (손실함수 : mse )
cost = tf.reduce_mean(tf.square(H-y))
'''
학습목적 : cost가 최소가 되는 W, b를 찾는 것
cost 함수는 2차함수이므로 곡선 그래프. 곡선위 미분값이 0이 되는 방향
(경사하강법GradientDescent)
'''
optimizer = tf.train.GradientDescentOptimizer(learning_rate=0.01)
train = optimizer.minimize(cost)
# 세션 생성
sess = tf.Session()
sess.run(tf.global_variables_initializer())
# 6000번 학습 (v2에서의 fit함수)
for step in range(6000):
    _, cost_val, W_val, b_val = sess.run([train, cost, W, b])
    if step%300==0:
        print("{}번째 cost:{}, W:{}, b:{}".format(step, cost_val, W_val, b_val))

0번째 cost:13.041595458984375, W:[-0.4498211], b:[0.9140647]
300번째 cost:0.0072715324349701405, W:[0.9011985], b:[1.2245991]
600번째 cost:0.0017157761612907052, W:[0.95200664], b:[1.1091003]
900번째 cost:0.00040485183126293123, W:[0.9766869], b:[1.0529962]
1200번째 cost:9.553165727993473e-05, W:[0.9886753], b:[1.0257437]
1500번째 cost:2.2543303202837706e-05, W:[0.9944987], b:[1.0125057]
1800번째 cost:5.318525381881045e-06, W:[0.99732786], b:[1.0060743]
2100번째 cost:1.2549868415590026e-06, W:[0.99870193], b:[1.0029507]
2400번째 cost:2.962651137750072e-07, W:[0.9993693], b:[1.0014337]
2700번째 cost:7.012523894900369e-08, W:[0.99969316], b:[1.0006975]
3000번째 cost:1.665967808150981e-08, W:[0.9998502], b:[1.00034]
3300번째 cost:4.048558821523329e-09, W:[0.9999265], b:[1.0001677]
3600번째 cost:9.734387740323314e-10, W:[0.9999638], b:[1.0000821]
3900번째 cost:2.646487473612069e-10, W:[0.99998134], b:[1.0000428]
4200번째 cost:8.733517381509515e-11, W:[0.9999891], b:[1.0000248]
4500번째 cost:8.733517381509515e-11, W:[0.99

In [18]:
# 최종적으로 나온 회귀식 H = W*x+b
W_, b_ = sess.run([W,b])

In [19]:
W_, b_

(array([0.9999891], dtype=float32), array([1.0000248], dtype=float32))

In [22]:
def predict(x):
    return W_[0] * x + b_[0]

In [23]:
predict(5)

5.999970257282257

## 2.2 predict을 위한 placeholder 이용
 - placeholder : 외부에서 데이터를 입력받을 수 있는 노드

In [21]:
x = tf.placeholder(dtype=tf.float32)
H = 1*x +1
sess = tf.Session()
sess.run([x, H], feed_dict={x:np.array([40,50])})

[array([40., 50.], dtype=float32), array([41., 51.], dtype=float32)]

In [24]:
# tensor 그래프 정의
# 데이터 셋 확보
x_data = np.array([1,2,3])
y_data = np.array([2,3,4])
# placeholder 설정(x, y)
x = tf.placeholder(dtype=tf.float32)
y = tf.placeholder(dtype=tf.float32)
# weight와 bias
W = tf.Variable(tf.random.normal([1]), name='weight')
b = tf.Variable(tf.random.normal([1]), name='bias')
# Hat, Hypothesis : 결과는 numpy 배열
H = W * x + b
# cost function (손실함수 : mse )
cost = tf.reduce_mean(tf.square(H-y))
'''
학습목적 : cost가 최소가 되는 W, b를 찾는 것
cost 함수는 2차함수이므로 곡선 그래프. 곡선위 미분값이 0이 되는 방향
(경사하강법GradientDescent)
'''
optimizer = tf.train.GradientDescentOptimizer(learning_rate=0.01)
train = optimizer.minimize(cost)
# 세션 생성
sess = tf.Session()
# W, b 초기화
sess.run(tf.global_variables_initializer())
# 6000번 학습 (v2 에서의 fit함수)
for step in range(6001):
    _, cost_val, W_val, b_val = sess.run([train, cost, W, b],
                                        feed_dict={x:x_data,
                                                   y:y_data})
    if step%300==0:
        print("{}번째 cost:{}, W:{}, b:{}".format(step, cost_val, W_val, b_val))

0번째 cost:0.37910428643226624, W:[0.25838655], b:[2.5697217]
300번째 cost:0.08583208173513412, W:[0.6605503], b:[1.771649]
600번째 cost:0.020252911373972893, W:[0.83511], b:[1.3748337]
900번째 cost:0.004778838250786066, W:[0.91990376], b:[1.1820776]
1200번째 cost:0.0011276003206148744, W:[0.9610929], b:[1.0884451]
1500번째 cost:0.00026607359177432954, W:[0.9811004], b:[1.0429633]
1800번째 cost:6.27836343483068e-05, W:[0.9908195], b:[1.0208696]
2100번째 cost:1.4815120266575832e-05, W:[0.99554044], b:[1.0101378]
2400번째 cost:3.495964165267651e-06, W:[0.9978336], b:[1.0049247]
2700번째 cost:8.251067242781573e-07, W:[0.9989476], b:[1.0023924]
3000번째 cost:1.9481819890643237e-07, W:[0.9994886], b:[1.0011625]
3300번째 cost:4.6209112980477585e-08, W:[0.9997508], b:[1.0005662]
3600번째 cost:1.0986809684254695e-08, W:[0.9998783], b:[1.0002761]
3900번째 cost:2.6936959329759702e-09, W:[0.99993986], b:[1.0001366]
4200번째 cost:6.699376053909134e-10, W:[0.9999702], b:[1.0000684]
4500번째 cost:1.5450041246367618e-10, W:[0.99998

In [25]:
# 예측하기
sess.run(H, feed_dict={x:5})

array([5.9999704], dtype=float32)

## 2.3 scale이 다른 데이터들의 회귀분석 구현(scale 조정X)

In [26]:
x_data = np.array([1, 2, 5, 8, 10])
y_data = np.array([5, 15, 68, 80, 95])
# placeholder 설정
x = tf.placeholder(dtype=tf.float32)
y = tf.placeholder(dtype=tf.float32)
# weight & bias
W = tf.Variable(tf.random.normal([1]))
b = tf.Variable(tf.random.normal([1]))
# Hypothesis
H = W * x + b
# cost (손실함수)
cost = tf.reduce_mean(tf.square(H-y))
# 경사하강법
optimizer = tf.train.GradientDescentOptimizer(learning_rate=0.0001)
train = optimizer.minimize(cost)
# Session & 변수 초기화
sess = tf.Session()
sess.run(tf.global_variables_initializer())
# 학습
for step in range(1, 6001):
    _, cost_val = sess.run([train, cost], feed_dict={x:x_data, y:y_data})
    if step%300==0:
        print("{}/6000번째 cost:{}".format(step+1, cost_val))

301/6000번째 cost:108.3859634399414
601/6000번째 cost:80.48823547363281
901/6000번째 cost:80.21051788330078
1201/6000번째 cost:80.17097473144531
1501/6000번째 cost:80.13475036621094
1801/6000번째 cost:80.0998306274414
2101/6000번째 cost:80.06611633300781
2401/6000번째 cost:80.03361511230469
2701/6000번째 cost:80.00224304199219
3001/6000번째 cost:79.97198486328125
3301/6000번째 cost:79.94276428222656
3601/6000번째 cost:79.9145736694336
3901/6000번째 cost:79.88737487792969
4201/6000번째 cost:79.86112976074219
4501/6000번째 cost:79.83583068847656
4801/6000번째 cost:79.8113784790039
5101/6000번째 cost:79.78779602050781
5401/6000번째 cost:79.76506042480469
5701/6000번째 cost:79.74310302734375
6001/6000번째 cost:79.7219467163086


## 2.4 scale이 다른 데이터들의 회귀분석 구현(scale 조정O)
### scale 조정방법 : 모든 데이터를 일정범위내로 조정
 - normalization(정규화) : 모든 데이터를 0~1 사이로 조정. 라이브러리 추천(sklearn.preprocessing.MinMaxScaler)
 - standarization(표준화) : 데이터를 평균을 0, 표준편차를 1로 조정. 라이브러리 추천(sklearn.preprocessing.StandardScaler) 

In [29]:
# 라이브러리 쓰지 않고 정규화
x_data = np.array([1, 2, 5, 8, 10])
y_data = np.array([5, 15, 68, 80, 95])
# x_data.min(), x_data.max(), x_data.mean(), x_data.std()
norm_scaled_x_data = (x_data - x_data.min()) / (x_data.max() - x_data.min())
norm_scaled_y_data = (y_data - y_data.min()) / (y_data.max() - y_data.min())
print(norm_scaled_x_data, norm_scaled_y_data)

[0.         0.11111111 0.44444444 0.77777778 1.        ] [0.         0.11111111 0.7        0.83333333 1.        ]


In [32]:
# 라이브러리 쓰지 않고 표준화
x_data = np.array([1, 2, 5, 8, 10])
y_data = np.array([5, 15, 68, 80, 95])
stan_scaled_x_data = (x_data - x_data.mean()) / x_data.std()
stan_scaled_y_data = (y_data - y_data.mean()) / y_data.std()
np.column_stack([x_data, norm_scaled_x_data, stan_scaled_x_data])

array([[ 1.        ,  0.        , -1.22474487],
       [ 2.        ,  0.11111111, -0.93313895],
       [ 5.        ,  0.44444444, -0.05832118],
       [ 8.        ,  0.77777778,  0.81649658],
       [10.        ,  1.        ,  1.39970842]])

In [33]:
np.column_stack([y_data, norm_scaled_y_data, stan_scaled_y_data])

array([[ 5.        ,  0.        , -1.32373476],
       [15.        ,  0.11111111, -1.04563922],
       [68.        ,  0.7       ,  0.42826713],
       [80.        ,  0.83333333,  0.76198177],
       [95.        ,  1.        ,  1.17912508]])